In [ ]:
using Stan, StanSample, DataFrames, CSV, Plots

In [ ]:
rawdata = CSV.read("../data/NLA2012_data.csv", DataFrame)
nothing

In [ ]:
data = Dict("n" => size(rawdata,1),
            "ne" => 8,
            "TN_logobs" => log.(rawdata.TN),
            "TP_logobs" => log.(rawdata.TP),
            "NANI" => rawdata.NANI,
            "NAPI" => rawdata.NAPI,
            "LF" => rawdata.forest_pct,
            "LW" => rawdata.wetland_pct,
            "Chla" => rawdata.Chla,
            "disc" => rawdata.discharge , # m^3/day
            "Vol" => rawdata.Vol,         # m^3
            "ecoN" => Int.(rawdata.eco_region .- 4),
            "trN" => Int.(rawdata.TSI),
            "WSArea" => rawdata.watershed_area
            )

In [ ]:
modelcode = open(f->read(f, String), "NLAmodel.stan");

In [ ]:
sm = SampleModel("NLA", modelcode);

In [ ]:
rc = stan_sample(sm, data=data);

In [ ]:
if success(rc)
    samples = read_samples(sm, :dataframe);
    sdf = read_summary(sm, :dataframe);
end
sdf[8:end,:]

In [ ]:
TN = [sdf[170:169+data["n"],2] sdf[170:169+data["n"],5] sdf[170:169+data["n"],7]]
TP = [sdf[170+data["n"]:169+data["n"]*2,2] sdf[170+data["n"]:169+data["n"]*2,5] sdf[170+data["n"]:169+data["n"]*2,7]]
Nload = [sdf[170+data["n"]*2:169+data["n"]*3,2] sdf[170+data["n"]*2:169+data["n"]*3,5] sdf[170+data["n"]*2:169+data["n"]*3,7]]
Pload = [sdf[170+data["n"]*3:169+data["n"]*4,2] sdf[170+data["n"]*3:169+data["n"]*4,5] sdf[170+data["n"]*3:169+data["n"]*4,7]]
nothing

In [ ]:
aN = sdf[40:47,2]
aP = sdf[48:55,2]
bN = sdf[56:63,2]
bP = sdf[64:71,2]
mN = sdf[72:79,2]
mP = sdf[80:87,2]
SN = sdf[88:95,2]
SP = sdf[96:103,2]
nothing

In [ ]:
EN_N = zeros(data["n"],1)
EN_P = zeros(data["n"],1)
DE_N = zeros(data["n"],1)
DE_P = zeros(data["n"],1)
for i in 1:data["n"]
    EN_N[i] = aN[data["ecoN"][i]]*data["Chla"][i]^bN[data["ecoN"][i]]/(data["Chla"][i]^bN[data["ecoN"][i]]+mN[data["ecoN"][i]]^bN[data["ecoN"][i]])/data["Vol"][i]
    EN_P[i] = aP[data["ecoN"][i]]*data["Chla"][i]^bP[data["ecoN"][i]]/(data["Chla"][i]^bP[data["ecoN"][i]]+mP[data["ecoN"][i]]^bP[data["ecoN"][i]])/data["Vol"][i]
    DE_N[i] = SN[data["ecoN"][i]]*TN[i,1]
    DE_P[i] = SP[data["ecoN"][i]]*TP[i,1]
end

In [ ]:
model_output = DataFrame(TNmean = TN[:,1], TN5 = TN[:,2], TN95 = TN[:,3],
                         TPmean = TP[:,1], TP5 = TP[:,2], TP95 = TP[:,3],
                         N_Load=Nload[:,1] ./ data["Vol"], 
                         N_EN = EN_N[:,1], 
                         N_DE = DE_N[:,1],
                         N_Outflow = data["disc"] ./ data["Vol"] .* TN[:,1],
                         P_Load=Pload[:,1] ./ data["Vol"], 
                         P_EN = EN_P[:,1], 
                         P_DE = DE_P[:,1], 
                         P_Outflow = data["disc"] ./ data["Vol"] .* TP[:,1]
)
nothing

In [ ]:
CSV.write("NLAmodel_output.csv", model_output)